# Computational Theory

In [16]:
import numpy as np
import matplotlib.pyplot as plt

## Problem 1: Binary Words and Operations

### Parity Function

The Parity Function is a systemmetric boolean function , meaning the order of 1s does not change the output. The parity function is typically used to classify a function as either even , odd or neither. In this case we are using a bitwise XOR operation , XOR is a logical operator repesented by "^" in Python. It only returns true if both inputs differ from each other e. g : 1 ^ 1 = 0 , false even though there are two truths. 

This function returns the bitwise XOR of three 32-bit integers.
We will use the numpy.int32 data type to ensure we meet the 32-bit word requirement for our SHA implenmentations,  SHA-256 , SHA-1 and SHA-224 all specifically require 32-bit words.

In our example we use : Parity(0b1100, 0b1010, 0b1001)

This is how the XOR is calculated



In [17]:
def Parity(x, y, z):
    """Return the bitwise parity (XOR) of three 32-bit integers.
       Formula: Parity(x, y, z) = x ⊕ y ⊕ z"""
    # See : https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32

    # Convert all inputs to unsigned 32-bit integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    # XOR all three together
    # The ^ operator in Python performs bitwise XOR
    return np.uint32(x ^ y ^ z)


In [18]:
def Ch(x,y,z):
    """Returns the bitwise choice function result for three 32-bit integers.

       The Ch (Choose) function selects bits from y or z based on x:
       - If x bit is 1 → choose the corresponding bit from y
       - If x bit is 0 → choose the corresponding bit from z

       Formula: Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)"""
    
    # Convert all inputs to unsigned 32-bit integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ ((~x) & z))


In [19]:
def Maj(x,y,z):
    """Return the bitwise majority function result for three 32-bit integers.
       The Maj (Majority) function returns the majority bit for each position:
       - If 2 or 3 inputs have bit=1 → output bit is 1
       - If 2 or 3 inputs have bit=0 → output bit is 0
    
       Formula: Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)"""
    
    # Convert all inputs to unsigned 32-bit integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [20]:
def ROTR(x , n):
    """Rotate right - bits wrap back around on the left"""
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))



def SHR(x, n):
    """SHR means regular shift (>>) with NO wrap-around.
       Bits that fall off are lost, zeros fill in from the left."""
    x = np.uint32(x)
    return np.uint32(x >> n)

In [21]:
def Sigma0(x):
    """Return the Sigma0 function used in SHA-256 for a 32-bit integer.
       This function uses THREE rotations (all bits wrap around):
       Formula: Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
       
       - No regular shifts (SHR) are used in uppercase Sigma functions"""
    
    # Ensure unsigned 32-bit requirement
    x = np.uint32(x)

    # Three rotations (all bits wrap around)
    rotr2 = ROTR(x, 2)
    rotr13 = ROTR(x, 13)
    rotr22 = ROTR(x, 22)

    # XOR all three together
    result = rotr2 ^ rotr13 ^ rotr22

    return np.uint32(result)



In [22]:
def Sigma1(x):
    """Return the Sigma1 function used in SHA-256 for a 32-bit integer.
       This function uses THREE rotations (all bits wrap around):
       Formula: Σ₁(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
       
       - No regular shifts (SHR) are used in uppercase Sigma functions"""
    
    # Ensure unsigned 32-bit requirement
    x = np.uint32(x)

    # Three rotations (all bits wrap around)
    rotr6 = ROTR(x, 6)
    rotr11 = ROTR(x, 11)
    rotr25 = ROTR(x, 25)

     # XOR all three together
    result = rotr6 ^ rotr11 ^ rotr25

    return np.uint32(result)


In [23]:
def sigma0(x):
    """Return the sigma0 function used in SHA-256 for a 32-bit integer.
       This function uses TWO rotations and ONE shift:
       Formula: σ₀(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)"""
    
    # Ensure unsigned 32-bit requirement
    x = np.uint32(x)

    # Two rotations (bits wrap around)
    rotr7 = ROTR(x, 7)
    rotr18 = ROTR(x, 18)

    # One regular shift (bits are lost - NO wrap around)
    shr3 = SHR(x,3) # Or simply x >> 3

    # XOR all three together
    result = rotr7 ^ rotr18 ^ shr3

    return np.uint32(result)


In [24]:
def sigma1(x):
    """Return the sigma1 function used in SHA-256 for a 32-bit integer.
       This function uses TWO rotations and ONE shift:
       Formula: σ₁(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)"""
    
    # Ensure unsigned 32-bit requirement
    x = np.uint32(x)

    # Two rotations (bits wrap around)
    rotr17 = ROTR(x, 17)
    rotr19 = ROTR(x, 19)

    # One regular shift (bits are lost - NO wrap around)
    shr10 = SHR(x,10) # Or simply x >> 10

    # XOR all three together
    result = rotr17 ^ rotr19 ^ shr10

    return np.uint32(result)


### Testing Problem 1 Functions

#### Testing Parity

The Parity function performs XOR on three inputs. We'll verify with:
- x = 0b1100 (12 in decimal)
- y = 0b1010 (10 in decimal)  
- z = 0b1001 (9 in decimal)

The XOR operation works step by step:

**Step 1: x ⊕ y**
```
   1100
⊕ 1010
------
   0110 (6 in decimal)
```

**Step 2: (x ⊕ y) ⊕ z**
```
   0110
⊕ 1001
------
   1111 (15 in decimal)
```



In [29]:
# Test Parity function
print("--- Testing Parity ---")
x, y, z = 0b1100, 0b1010, 0b1001

# Show inputs
print(f"x = {x:04b} ({x})")
print(f"y = {y:04b} ({y})")
print(f"z = {z:04b} ({z})")

# Calculate result
result = Parity(x, y, z)
print(f"\nParity(x, y, z) = {result:04b} ({result})")

# Verify
expected = 15
assert result == expected, f"Test failed, expected {expected}, got {result}"
print(f"Test passed , result matches expected value of {expected}\n")

--- Testing Parity ---
x = 1100 (12)
y = 1010 (10)
z = 1001 (9)

Parity(x, y, z) = 1111 (15)
Test passed , result matches expected value of 15



#### Testing Ch (Choose)

The Ch function chooses bits from y or z based on x:
- If x bit = 1 → choose from y
- If x bit = 0 → choose from z

We'll use the same inputs as before(x=1100, y=1010, z=1001)
Expected result: 1001 (9 in decimal)

In [30]:
# Test Ch function
print("--- Testing Ch (Choose) ---")
x, y, z = 0b1100, 0b1010, 0b1001

# Show inputs
print(f"x = {x:04b} ({x}) - selector")
print(f"y = {y:04b} ({y}) - first option")
print(f"z = {z:04b} ({z}) - second option")

# Calculate result
result = Ch(x, y, z)
print(f"\nCh(x, y, z) = {result:04b} ({result})")


# Verify
expected = 9
assert result == expected, f"Test failed: expected {expected}, got {result}"
print(f"\nTest passed, the result matches expected value of {expected}\n")

--- Testing Ch (Choose) ---
x = 1100 (12) - selector
y = 1010 (10) - first option
z = 1001 (9) - second option

Ch(x, y, z) = 1001 (9)

Test passed, the result matches expected value of 9



#### Testing Maj (Majority)

The Maj function returns the majority bit at each position.

Using the same inputs (x=1100, y=1010, z=1001)
Expected result: 1000 (8 in decimal)

In [31]:
# Test Maj function
print("-- Testing Maj (Majority) ---")
x, y, z = 0b1100, 0b1010, 0b1001

# Show inputs
print(f"x = {x:04b} ({x})")
print(f"y = {y:04b} ({y})")
print(f"z = {z:04b} ({z})")

# Calculate result
result = Maj(x, y, z)
print(f"\nMaj(x, y, z) = {result:04b} ({result})")


# Verify
expected = 8
assert result == expected, f"Test failed, expected {expected}, got {result}"
print(f"\nTest passed, result matches expected value of {expected}\n")

-- Testing Maj (Majority) ---
x = 1100 (12)
y = 1010 (10)
z = 1001 (9)

Maj(x, y, z) = 1000 (8)

Test passed, result matches expected value of 8



#### Testing Sigma Functions

The Sigma functions combine multiple rotations (and shifts for lowercase sigma).
We'll verify they execute without errors and produce consistent results.


In [33]:
# Test all Sigma functions
print("--- Testing Sigma Functions ---")
test_value = 0b1100

print(f"Test value: {test_value:032b} ({test_value})\n")

# Test Sigma0
result_Sigma0 = Sigma0(test_value)
print(f"Sigma0({test_value}):  {result_Sigma0:032b} ({result_Sigma0})")

# Test Sigma1
result_Sigma1 = Sigma1(test_value)
print(f"Sigma1({test_value}):  {result_Sigma1:032b} ({result_Sigma1})")

# Test sigma0
result_sigma0 = sigma0(test_value)
print(f"sigma0({test_value}):  {result_sigma0:032b} ({result_sigma0})")

# Test sigma1
result_sigma1 = sigma1(test_value)
print(f"sigma1({test_value}):  {result_sigma1:032b} ({result_sigma1})")

print("\nAll Sigma functions executed.")

--- Testing Sigma Functions ---
Test value: 00000000000000000000000000001100 (12)

Sigma0(12):  00000000011000000011000000000011 (6303747)
Sigma1(12):  00110001100000000000011000000000 (830473728)
sigma0(12):  00011000000000110000000000000001 (402849793)
sigma1(12):  00000000000001111000000000000000 (491520)

All Sigma functions executed.


## Problem 2: Fractional Parts of Cube Roots

### Primes(n) Function


## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords

## End